# Flat10 runs with FaIR v2.1.2

In [ ]:
import os
import fair
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch
from tqdm.auto import tqdm

In [ ]:
fair.__version__

In [ ]:
f = FAIR()
f.define_time(0, 320, 1)

In [ ]:
scenarios = ['esm-flat10', 'esm-flat10_zec', 'esm-flat10_cdr']
f.define_scenarios(scenarios)

In [ ]:
cal_1_2_0_file = pooch.retrieve(
    url = "https://zenodo.org/records/8399112/files/calibrated_constrained_parameters.csv",
    known_hash = "md5:de3b83432b9d071efdd1427ad31e9076"
)

In [ ]:
cal_1_2_0_df = pd.read_csv(cal_1_2_0_file, index_col=0)
cal_1_2_0_df

In [ ]:
f.define_configs(list(cal_1_2_0_df.index))

In [ ]:
species = ['CO2', 'CH4', 'N2O']
properties = {
    "CO2": {
        'type': 'co2',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "CH4": {
        'type': 'ch4',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "N2O": {
        'type': 'n2o',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    }
}

In [ ]:
# declare species and properties
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
# fill emissions: zero for non-CO2
f.emissions.loc[dict(specie="CH4")] = 0
f.emissions.loc[dict(specie="N2O")] = 0

In [ ]:
# constant pre-industrial concentration for non-CO2 GHGs
f.concentration.loc[dict(specie='CH4')] = 808.2490285
f.concentration.loc[dict(specie='N2O')] = 273.021047

In [ ]:
np.linspace(9.9, -9.9, 100)  # 100 timepoints, from years 100.5 to 199.5 inclusive

In [ ]:
# fill emissions of CO2 for each scenario
f.emissions.loc[dict(specie="CO2", scenario="esm-flat10")] = 10 * 44.009 / 12.011
f.emissions.loc[dict(specie="CO2", scenario="esm-flat10_zec", timepoints=np.arange(0.5, 100))] = 10 * 44.009 / 12.011
f.emissions.loc[dict(specie="CO2", scenario="esm-flat10_zec", timepoints=np.arange(100.5, 320))] = 0
f.emissions.loc[dict(specie="CO2", scenario="esm-flat10_cdr", timepoints=np.arange(0.5, 100))] = 10 * 44.009 / 12.011
f.emissions.loc[dict(specie="CO2", scenario="esm-flat10_cdr", timepoints=np.arange(100.5, 200))] = np.linspace(9.9, -9.9, 100)[:, None] * 44.009 / 12.011
f.emissions.loc[dict(specie="CO2", scenario="esm-flat10_cdr", timepoints=np.arange(200.5, 300))] = -10 * 44.009 / 12.011
f.emissions.loc[dict(specie="CO2", scenario="esm-flat10_cdr", timepoints=np.arange(300.5, 320))] = 0

In [ ]:
# Get default species configs
f.fill_species_configs()

# Climate response
fill(f.climate_configs['ocean_heat_capacity'], cal_1_2_0_df.loc[:,'clim_c1':'clim_c3'])
fill(f.climate_configs['ocean_heat_transfer'], cal_1_2_0_df.loc[:,'clim_kappa1':'clim_kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], cal_1_2_0_df.loc[:,'clim_epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], cal_1_2_0_df.loc[:,'clim_gamma'])
fill(f.climate_configs['stochastic_run'], False)

# carbon cycle
fill(f.species_configs['iirf_0'], cal_1_2_0_df.loc[:, 'cc_r0'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_airborne'], cal_1_2_0_df.loc[:, 'cc_rA'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_uptake'], cal_1_2_0_df.loc[:, 'cc_rU'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_temperature'], cal_1_2_0_df.loc[:, 'cc_rT'].values.squeeze(), specie='CO2')

# Scale CO2 forcing based on its 4xCO2 calibration
fill(f.species_configs["forcing_scale"], cal_1_2_0_df["fscale_CO2"].values.squeeze(), specie='CO2')

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f.species_configs['baseline_concentration'], 284.3169988, specie='CO2')
fill(f.species_configs['baseline_concentration'], 808.2490285, specie='CH4')
fill(f.species_configs['baseline_concentration'], 273.021047, specie='N2O')

In [ ]:
f.species_configs

In [ ]:
# set initial conditions
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

In [ ]:
f.run()

In [ ]:
fig, ax = pl.subplots(2, 2)
ax[0,0].plot(f.timepoints[:150], f.emissions.loc[dict(specie="CO2", scenario="esm-flat10", timepoints=np.arange(0.5, 150))], color='k', alpha=0.1);
ax[0,0].plot(f.timepoints, f.emissions.loc[dict(specie="CO2", scenario="esm-flat10_zec")], color='b', alpha=0.1);
ax[0,0].plot(f.timepoints, f.emissions.loc[dict(specie="CO2", scenario="esm-flat10_cdr")], color='r', alpha=0.1, ls=':');
ax[0,1].plot(np.arange(0, 151), f.cumulative_emissions.loc[dict(specie="CO2", scenario="esm-flat10", timebounds=np.arange(0, 151))], color='k', alpha=0.1);
ax[0,1].plot(f.cumulative_emissions.loc[dict(specie="CO2", scenario="esm-flat10_zec")], color='b', alpha=0.1);
ax[0,1].plot(f.cumulative_emissions.loc[dict(specie="CO2", scenario="esm-flat10_cdr")], color='r', alpha=0.1, ls=':');
ax[1,0].plot(np.arange(0, 151), f.concentration.loc[dict(specie="CO2", scenario="esm-flat10", timebounds=np.arange(0, 151))], color='k', alpha=0.1);
ax[1,0].plot(f.concentration.loc[dict(specie="CO2", scenario="esm-flat10_zec")], color='b', alpha=0.1);
ax[1,0].plot(f.concentration.loc[dict(specie="CO2", scenario="esm-flat10_cdr")], color='r', alpha=0.1, ls=':');
ax[1,1].plot(np.arange(0, 151), f.temperature.loc[dict(layer=0, scenario="esm-flat10", timebounds=np.arange(0, 151))], color='k', alpha=0.1);
ax[1,1].plot(f.temperature.loc[dict(layer=0, scenario="esm-flat10_zec")], color='b', alpha=0.1);
ax[1,1].plot(f.temperature.loc[dict(layer=0, scenario="esm-flat10_cdr")], color='r', alpha=0.1, ls=':');

In [ ]:
# TCRE is just warming at year 100!
tcre = f.temperature.loc[dict(layer=0, scenario="esm-flat10", timebounds=100)]
pl.hist(tcre)

In [ ]:
# ZEC50 is just warming at year 150 minus year 100!
zec50 = f.temperature.loc[dict(layer=0, scenario="esm-flat10_zec", timebounds=150)] - f.temperature.loc[dict(layer=0, scenario="esm-flat10_zec", timebounds=100)]
pl.hist(zec50)

In [ ]:
# ZEC100 is just warming at year 200 minus year 100!
zec100 = f.temperature.loc[dict(layer=0, scenario="esm-flat10_zec", timebounds=200)] - f.temperature.loc[dict(layer=0, scenario="esm-flat10_zec", timebounds=100)]
pl.hist(zec100)

In [ ]:
# ZEC200 is just warming at year 300 minus year 100!
zec200 = f.temperature.loc[dict(layer=0, scenario="esm-flat10_zec", timebounds=300)] - f.temperature.loc[dict(layer=0, scenario="esm-flat10_zec", timebounds=100)]
pl.hist(zec200)

In [ ]:
# TNZ can be calculated as a 20 year average around year 150 in esm-flat10-cdr minus a 20 year average around year 125 in esm-flat10
tnz = f.temperature.loc[dict(layer=0, scenario="esm-flat10_cdr", timebounds=150)] - f.temperature.loc[dict(layer=0, scenario="esm-flat10", timebounds=125)]
pl.hist(tnz)

In [ ]:
# TR1000 can be calculated as a 20 year average around year 200 in esm-flat10-cdr minus a 20 year average around year 100 in esm-flat10
tr1000 = f.temperature.loc[dict(layer=0, scenario="esm-flat10_cdr", timebounds=200)] - f.temperature.loc[dict(layer=0, scenario="esm-flat10", timebounds=100)]
pl.hist(tr1000)

In [ ]:
# TR0 can be calculated as a 20 year average around year 310 in esm-flat10-cdr
tr0 = f.temperature.loc[dict(layer=0, scenario="esm-flat10_cdr", timebounds=310)]
pl.hist(tr0)

In [ ]:
# Time to Peak Warming (tPW) can be calculated as the time difference between the peak value of 20-year smoothed global mean temperatures and the point that net zero is achieved in esm-flat10-cdr (year 150)
tpw = f.temperature.loc[dict(layer=0, scenario="esm-flat10_cdr")].argmax(axis=0) - 150
pl.hist(tpw)

In [ ]:
df = pd.DataFrame(
    {
        "tcre": tcre,
        "zec50": zec50,
        "zec100": zec100,
        "zec200": zec200,
        "tr1000": tr1000,
        "tr0": tr0,
        "tpw": tpw,
    },
    index = f.configs
)

In [ ]:
os.makedirs('../output/', exist_ok=True)
df.to_csv('../output/flat10_key-metrics_fair2.1.2_cal1.2.0.csv')